# Finding Nice Anniversaries
I want to systematically find all nice days that mark some sort of "meaningful" number of years in our relationship. E.g. 3.1415 ($\approx\pi$) years. Formally speaking, that is looking for two numbers $\alpha$ and $\beta$ along with corresponding integer factors $p$ and $q$ that approximate the number of years I am together with Rosalie up to one day. $\alpha$ and $\beta$ are those "special numbers":

$$
\alpha, \beta \in \left\{ e^z, \pi^z, \delta^z, \phi^z \vert z \in \mathbb{Q} \right\}
$$

So, if $x = \operatorname{days}(B)\, / \, 365.2425$ is the number of days we are together then the formal criterion is

$$
\left| x - \frac{p\cdot\alpha}{q\cdot\beta} \right| < \frac{1}{365.2425}
$$

But I will compute

$$
\operatorname{round}\left( \frac{x\cdot\beta}{\alpha}, 3 \right) = t
$$

and then find $p$ and $q$ numerically so that $t = p\, / \, q$. Although actually that thing with the rounding... might be wrong, but so far it works...

In [1]:
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math, Markdown

In [2]:
# begin of love
begin_date = dt.datetime(year=2016, month=3, day=30)

# precision I need
prec = 1. / 365.2425

# important numbers (more can be added of course)
pi = np.pi
print(f" pi = {pi} (Kreiszahl)")
e = np.exp(1)
print(f"  e = {e} (Eulersche Zahl)")
d = 4.669201609102990671853203821578
print(f"  d = {d} (Feigenbaum constant)")
phi = ( 1 + np.sqrt(5) ) / 2
print(f"phi = {phi} (goldener Schnitt)")

# create arrays of powers of those important numbers
powers = np.array([1, 2, 3])
PI = pi**powers
E = e**powers
D = d**powers
PHI = phi**powers

# collecting the numbers and their text representations
nums = np.array([pi, e, phi, 1])
nums_txt = ["pi", "e", "1"]
nums_ltx = [r"\pi", "e", ""]
NUMS = np.concatenate((PI, E, [1]))

 pi = 3.141592653589793 (Kreiszahl)
  e = 2.718281828459045 (Eulersche Zahl)
  d = 4.66920160910299 (Feigenbaum constant)
phi = 1.618033988749895 (goldener Schnitt)


## Version A
Uses modulo and integer numbers. Not very elegant, loads of "IFs" and 3 loops. Also it spits out tons of duplicates.

In [121]:
x = (dt.datetime.now() - begin_date).days / 365.2425

idx_list = []

for a, alpha in enumerate(NUMS):
    for b, beta in enumerate(NUMS):
        for q in range(1,10):
            if (a == b):
                pass
            elif (int(x * q * beta * 365.2425) % int(alpha * 365.2425) == 0):
                idx_list.append((a,b,q, int(x * q * beta * 365.2425) // int(alpha * 365.2425)))

## Version B
Much better! Only 2 loops and way more analytical. It directly finds the two integers $p$ and $q$ that would be necessary to approximate

$$
x\cdot\frac{\beta}{\alpha} \approx \frac{p}{q}
$$

sufficiently well. 

In [4]:
# small helper functions for the text formatting
pwr_ltx = lambda pwr : "" if pwr==1 else f"^{pwr}"
show_int = lambda i : "" if i==1 else f"{i}"

delta = (dt.datetime.now() - begin_date).days
special_dates = []

# loop through days from today to next year
for x in np.linspace(delta, delta+365, 365+1, dtype=float):
    
    # loop over the candidate numbers (enumerator)
    for a, alpha in enumerate(NUMS):
        
        # loop over the candidate numbers (denominator)
        for b, beta in enumerate(NUMS):
            
            # approximate this number by a rational fraction
            tmp = (x/365.2425) * beta / alpha
            int_ratio = round(tmp, 3).as_integer_ratio()
            
            # from here it's basically just generating the LaTeX text
            alpha_base_txt = nums_txt[a // len(powers)]
            alpha_base_ltx = nums_ltx[a // len(powers)]
            alpha_power = a % len(powers) + 1
            
            beta_base_txt = nums_txt[b // len(powers)]
            beta_base_ltx = nums_ltx[b // len(powers)]
            beta_power = b % len(powers) + 1
            
            diff = np.abs((x / 365.2425) - int_ratio[0]*alpha / (int_ratio[1]*beta)) * 365.2425
            # if the approximation is close enough...
            if np.all(np.greater(10, int_ratio)) and alpha_base_txt != beta_base_txt and diff < 1.:
                txt = f"{int_ratio[0]} * {alpha_base_txt}^{alpha_power} / {int_ratio[1]} * {beta_base_txt}^{beta_power}"
                
                if (beta == 1.) & (int_ratio[1] == 1):
                    latex = f"${show_int(int_ratio[0])}{alpha_base_ltx}{pwr_ltx(alpha_power)}$"
                elif beta == 1.:
                    latex = (r"$\frac{" + 
                             f"{show_int(int_ratio[0])}{alpha_base_ltx}{pwr_ltx(alpha_power)}" + 
                             r"}{" + 
                             f"{show_int(int_ratio[1])}" + 
                             r"}$")
                else:  
                    latex = (r"$\frac{" + 
                             f"{show_int(int_ratio[0])}{alpha_base_ltx}{pwr_ltx(alpha_power)}" + 
                             r"}{" + 
                             f"{show_int(int_ratio[1])}{beta_base_ltx}{pwr_ltx(beta_power)}" +
                             r"}$")
                
                # ...append it to the list of nice anniversaries
                special_dates.append(((begin_date + dt.timedelta(days=x)).strftime("%-d. %b %Y"), x / 365.2425, diff, latex))

Now that I have all that computed and collected in a list, I only need to put it into a Markdown file, so I can compile an HTML from it and print that to PDF.

In [8]:
md_string = """
| Date | Value [Years] | Difference [Days] | Display | Rating |
| :--- | ------------: | ----------------: | ------: | -----: |
"""

for sd in special_dates:
    md_string += f"""| {sd[0]} | {sd[1]:.2f} | {sd[2]:.2f} | {sd[3]} |  |\n"""

Let's have a look at it

In [9]:
display(Markdown(md_string))


| Date | Value [Years] | Difference [Days] | Display | Rating |
| :--- | ------------: | ----------------: | ------: | -----: |
| 18. Dec 2020 | 4.72 | 0.23 | $\frac{9\pi^3}{8e^2}$ |  |
| 31. Dec 2020 | 4.76 | 0.46 | $\frac{7e}{4}$ |  |
| 14. Jan 2021 | 4.79 | 0.36 | $\frac{3e^3}{4\pi}$ |  |
| 15. Jan 2021 | 4.80 | 0.64 | $\frac{3e^3}{4\pi}$ |  |
| 6. Mar 2021 | 4.93 | 0.40 | $\frac{\pi^2}{2}$ |  |
| 7. Mar 2021 | 4.94 | 0.60 | $\frac{\pi^2}{2}$ |  |
| 7. Apr 2021 | 5.02 | 0.02 | $\frac{e^3}{4}$ |  |
| 8. Apr 2021 | 5.02 | 0.98 | $\frac{e^3}{4}$ |  |
| 1. May 2021 | 5.09 | 0.25 | $\frac{5e^3}{2\pi^2}$ |  |
| 26. Jun 2021 | 5.24 | 0.12 | $\frac{7e^2}{\pi^2}$ |  |
| 28. Jun 2021 | 5.25 | 0.19 | $\frac{5\pi^3}{4e^2}$ |  |
| 15. Jul 2021 | 5.29 | 0.13 | $\frac{9e^2}{4\pi}$ |  |
| 6. Sep 2021 | 5.44 | 0.34 | $2e$ |  |
| 9. Sep 2021 | 5.45 | 0.20 | $\frac{3\pi^2}{2e}$ |  |
| 28. Sep 2021 | 5.50 | 0.03 | $\frac{7\pi}{4}$ |  |
| 14. Oct 2021 | 5.54 | 0.10 | $\frac{3e^2}{4}$ |  |
| 15. Oct 2021 | 5.54 | 0.90 | $\frac{3e^2}{4}$ |  |
| 2. Nov 2021 | 5.59 | 0.26 | $\frac{7e^3}{8\pi}$ |  |
| 3. Nov 2021 | 5.60 | 0.74 | $\frac{7e^3}{8\pi}$ |  |
| 12. Dec 2021 | 5.70 | 0.08 | $\frac{\pi^3}{2e}$ |  |
| 13. Dec 2021 | 5.71 | 0.92 | $\frac{\pi^3}{2e}$ |  |


Finally, write it to disk

In [10]:
file = open("/home/rmnldwg/OneDrive/Git/rmnldwg/anniversary/table.md", 'w')
file.write(md_string)
file.close()